Recurrent neural networks are used when a data
pattern changes over time. RNNs can be assumed as 
unrolled over time.

RNNs have feedback loops in which the output from the previous firing or time index T is fed as one of the inputs at time index T+1. There might be cases in which the output of the neuron is fed to itself as input. These are well suited for applications involving sequences.

Encoding RNN- Enables network to take an input of the sequence form.

Generating RNN- network basically outputs a sequence of numbers or values, like words in a sentence.

General recurrent neural networks - combination of the preceding two types of RNNs. It's used to generate sequences and, thus, are widely used in NLG (natural language generation) tasks.

# encoder - decoder networks
This uses one network to create an internal representation of the input, or to 'encode' it, and that representation is used as an input for another network to produce the output.

## recursive neural networks
In a recursive neural network, a fixed set of weights is recursively applied onto the network structure and is primarily used to discover, the hierarchy or structure of the data. 

In [7]:
import numpy as np
name = [1,2,3,4,5]
name1 = [1,2.2,4,1,1]
[np.round(i)==j for i,j in zip(name1, name)]

[True, True, False, False, False]

# word vector representations

#### Introduction to word Embedding
#### Neural Language Model

The feedforward neural net language model (FNNLM) proposed by Bengio introduces a feedforward neural network consisting of a single hidden layer than prdicts the future words. 
1. Embedding layer: This keeps a record of the representation of all the words in the training dataset. It is initialized with a set of random weights. The embedding layer is made up of three parts, which includes the size of the vocabulary, the output size of the vector in which all the words will be embedded, and the length of the input sequences to the model.

# word2vec code

In [1]:
import random, collections, math, os, zipfile, time, re
import numpy as np
import tensorflow as tf

In [2]:
from matplotlib import pylab
%matplotlib inline

from six.moves import range
from six.moves.urllib.request import urlretrieve

"""make sure the dataset link is copied correctly"""

dataset_link = 'http://mattmahoney.net/dc/'
zip_file = 'text8.zip'

In [3]:
def data_download(zip_file):
    """downloading the required file"""
    if not os.path.exists(zip_file):
        zip_file, _ = urlretrieve(dataset_link+zip_file, zip_file)
        print("file downloaded successfully")
        
    return None
data_download(zip_file)

In [4]:
"""Extracting the dataset in separate folder"""
extracted_folder = 'dataset'
if not os.path.isdir(extracted_folder):
    with zipfile.ZipFile(zip_file) as zf:
        zf.extractall(extracted_folder)
with open('dataset/text8') as ft_:
    full_text = ft_.read()

In [5]:
def text_processing(ft8_text):
    """Replacing punctuation marks with tokens"""
    ft8_text = ft8_text.lower()
    ft8_text = ft8_text.replace('.', ' <period> ')
    ft8_text = ft8_text.replace(',', ' <comma> ')
    ft8_text = ft8_text.replace('"', ' <quotation> ')
    ft8_text = ft8_text.replace(';', ' <semicolon> ')
    ft8_text = ft8_text.replace('!', ' <exclamation> ')
    ft8_text = ft8_text.replace('?', ' <question> ')
    ft8_text = ft8_text.replace('(', ' <paren_l> ')
    ft8_text = ft8_text.replace(')', ' <paren_r> ')
    ft8_text = ft8_text.replace('--', ' <hyphen> ')
    ft8_text = ft8_text.replace(':', ' <colon> ')
    ft8_text_tokens = ft8_text.split()
    return ft8_text_tokens

ft_tokens = text_processing(full_text)

In [6]:
"""Shortlisting words with frequency more than 7"""
word_cnt = collections.Counter(ft_tokens)
shortlisted_words = [w for w in ft_tokens if word_cnt[w] > 7 ]

In [7]:
print(shortlisted_words[:15])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including']


# CBow Model

The continuous bag-of-words mode l shares an architectural similarity to the FNNLM.

### Subsampling frequent words
### Negative sampling

Negative sampling is a simplified form of the noise contrastive estimation (NCE) approach, as it makes certain assumptions while selecting the count of the noise, or negative, samples and their distribution.

# Skip gram

In [8]:
"""Creating the threshold and performing the subsampling"""
thresh = 0.00005
word_counts = collections.Counter(word_cnt)
total_count = len(word_cnt)
freqs = {word: count / total_count for word, count in word_counts.items()}
p_drop = {word: 1 - np.sqrt(thresh/freqs[word]) for word in
word_counts}
train_words = [word for word in word_cnt if p_drop[word] <
random.random()]

In [14]:
len(train_words)

235478

In [9]:
def skipG_target_set_generation(batch_, batch_index, word_window):
    """The function combines the words of given word_window
size next to the index, for the SkipGram model"""
    random_num = np.random.randint(1, word_window+1)
    words_start = batch_index - random_num if (batch_index -
random_num) > 0 else 0
    words_stop = batch_index + random_num
    window_target = set(batch_[words_start:batch_index] +
batch_[batch_index+1:words_stop+1])
    return list(window_target)

In [10]:
def skipG_batch_creation(short_words, batch_length, word_window):
    """The function internally makes use of the skipG_target_
set_generation() function and combines each of the label
    words in the shortlisted_words with the words of word_
window size around"""
    batch_cnt = len(short_words)//batch_length
    short_words = short_words[:batch_cnt*batch_length]  
    for word_index in range(0, len(short_words), batch_length):
        input_words, label_words = [], []
        word_batch = short_words[word_index:word_index+batch_length]
        for index_ in range(len(word_batch)):
            batch_input = word_batch[index_]
            batch_label = skipG_target_set_generation(word_batch, index_, word_window)
            # Appending the label and inputs to the initial
#list. Replicating input to the size of labels in the window
            label_words.extend(batch_label)
            input_words.extend([batch_input]*len(batch_label))
            yield input_words, label_words
        

In [18]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

tf_graph = tf.Graph()
with tf_graph.as_default():
    input_ = tf.placeholder(tf.int32, [None], name='input_')
    label_ = tf.placeholder(tf.int32, [None, None],name='label_')

In [20]:
tf.__version__

'2.0.0'

In [22]:
with tf_graph.as_default():
    word_embed = tf.Variable(tf.random_uniform((len(rev_dictionary),300), -1,1))
    embedding = tf.nn.embedding_lookup(word_embed, input_)

NameError: name 'rev_dictionary' is not defined